Rodar no ambiente conda já criado bioinfo2

In [ ]:
# baixando o arquivo de trabalho
wget ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20130502/ALL.chr22.phase3_shapeit2_mvncall_integrated_v5b.20130502.genotypes.vcf.gz

In [ ]:
# baixando o metadado
wget ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20130502/integrated_call_samples_v3.20130502.ALL.panel

In [ ]:
# baixando o arquivo tabix
wget ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20130502/ALL.chr22.phase3_shapeit2_mvncall_integrated_v5b.20130502.genotypes.vcf.gz.tbi

In [ ]:
# Selecionar 30 amostras aleatórias (15 casos + 15 controles)
shuf integrated_call_samples_v3.20130502.ALL.panel | head -n 30 | awk '{print $1}' > samples.txt

# Criar arquivo de fenótipos
awk 'NR<=15 {print $1, $1, 2} NR>15 {print $1, $1, 1}' samples.txt > pheno.txt

In [ ]:
# filtrar vcf MAF > 0.05% para dar poder estatistico
bcftools view -S samples.txt -i 'MAF[0] > 0.0005' -Oz -o chr22_filtered.vcf.gz ALL.chr22.phase3_shapeit2_mvncall_integrated_v5b.20130502.genotypes.vcf.gz

In [ ]:
###codigo adicional para gerar o arquivo 'sample.txt' apartir do arquivo 'pheno.txt'
cut -d' ' -f1 dia_pheno.txt > dia_pheno_clean.txt


In [ ]:
# Converter para PLINK e Filtros Adicionais
plink --vcf chr22_filtered.vcf.gz --make-bed --maf 0.05 --geno 0.05 --hwe 1e-6 --allow-extra-chr --out chr22_clean
plink --vcf ad_wxs_15filtered.vcf.gz --make-bed --maf 0.05 --geno 0.05 --hwe 1e-6 --allow-extra-chr --out ad_wxs_clean
plink --vcf dia_wxs_15filtered.vcf.gz --make-bed --double-id --maf 0.05 --geno 0.05 --hwe 1e-6 --allow-extra-chr --out dia_wxs_clean

In [ ]:
# rodar o GWAS
plink --bfile chr22_clean --pheno pheno.txt --assoc --allow-no-sex --out gwas_results
plink --bfile ad_wxs_clean --pheno ad_pheno.txt --assoc --allow-no-sex --out gwas_ad_results
plink --bfile dia_wxs_clean --pheno dia_pheno.txt --assoc --allow-no-sex --out gwas_dia_results

SyntaxError: invalid syntax (<ipython-input-1-e5e51f4c96b0>, line 2)

In [ ]:
# conferindo os resultados:
#1. Verifique as primeiras linhas do arquivo:
head -n 5 gwas_results.assoc
#2. Verifique os nomes das colunas:
awk 'NR==1 {print; exit}' gwas_results.assoc
#3. Conte o número de SNPs:
wc -l gwas_results.assoc  # Total de linhas
awk 'NR>1' gwas_results.assoc | wc -l  # SNPs apenas (excluindo cabeçalho)
#4. Verifique a distribuição dos valores-p:
# Para valores-p não missing:
awk '$9 != "NA" && NR>1 {print $9}' gwas_results.assoc | sort -g | uniq -c | head
# Verifique valores problematicos
# Valores não numéricos na coluna P (geralmente coluna 9):
awk 'NR>1 && $9 !~ /^[0-9.eE-]+$/ {print $9}' gwas_results.assoc | sort | uniq -c

In [ ]:
# gerando script em R para gerar os graficos:
nano generate_plots_fixed.R

In [ ]:
#!/usr/bin/env Rscript

# Instalar pacote se necessário
if (!require("qqman")) {
    install.packages("qqman", repos="https://cloud.r-project.org")
    library(qqman)
}

# Ler dados
gwas_data <- read.table("gwas_dia_results.assoc", header=TRUE)

# Criar IDs únicos para SNPs (já que a coluna SNP tem ".")
gwas_data$SNP_ID <- paste0("chr", gwas_data$CHR, ":", gwas_data$BP)

# Verificar e limpar valores-p
gwas_data$P <- as.numeric(gwas_data$P)
valid_data <- gwas_data[!is.na(gwas_data$P) & is.finite(gwas_data$P), ]

# 1. Manhattan Plot
png("manhattan_dia_final.png", width=1200, height=600)
manhattan(valid_data,
          chr = "CHR",
          bp = "BP",
          p = "P",
          snp = "SNP_ID",  # Usar IDs criados
          main = "Manhattan Plot - GWAS Results",
          suggestiveline = -log10(1e-5),
          genomewideline = -log10(5e-8))
dev.off()

# 2. QQ Plot
png("qq_dia_final.png", width=600, height=600)
qq(valid_data$P, main = "QQ Plot - GWAS Results")
dev.off()

print("Gráficos gerados com sucesso! Verifique: manhattan_final.png e qq_final.png")

SyntaxError: invalid syntax (<ipython-input-3-e4928181f3b4>, line 4)

In [ ]:
Rscript generate_plots_fixed.R

In [ ]:
# 1. SNPs com p < 0.05 (significativos)
awk 'NR==1 || $9 < 0.05' gwas_results.assoc > snps_significativos.txt
awk 'NR==1 || $9 < 0.05' gwas_ad_results.assoc > snps_ad_significativos.txt
awk 'NR==1 || $9 < 0.05' gwas_dia_results.assoc > snps_dia_significativos.txt

# 2. SNPs com significância genômica (p < 5e-8)
awk 'NR==1 || $9 < 5e-8' gwas_results.assoc > snps_genomicos.txt

# Apartir daqui prepararemos para analise de pleiotropia

In [ ]:
# Passo 1: Executar o GWAS para ambas as doenças
# Doença 1 (exemplo)
plink --bfile chr22_filtered \
      --pheno pheno_doenca1.txt \
      --assoc \
      --out gwas_doenca1

# Doença 2
plink --bfile chr22_filtered \
      --pheno pheno_doenca2.txt \
      --assoc \
      --out gwas_doenca2

In [ ]:
# Passo 2: Identificar SNPs significativos em ambas as doenças
# SNPs com p < 0.05 em ambas (ajuste o limiar conforme necessário)
awk 'NR==FNR && $9 < 0.05 {print $2; next}
     NR!=FNR && $9 < 0.05 && ($2 in snps) {print $2}' \
     gwas_ad_results.assoc gwas_dia_results.assoc > snps_pleiotropicos.txt


In [ ]:
#Passo 3: Análise estatística de pleiotropia
#Opção 1: Teste de heterogeneidade (Cochran's Q)
# Combinar resultados (requer colunas: SNP, P_DOENCA1, P_DOENCA2)
paste <(awk 'NR>1 {print $2, $9}' gwas_ad_results.assoc) \
      <(awk 'NR>1 {print $9}' gwas_dia_results.assoc) > combined_pvalues.txt
# remover linhas que não tenham 3 colunas
# Criar versão filtrada (mantém apenas linhas com 3 colunas)
awk 'NF==3' combined_pvalues.txt > combined_pvalues_filtered.txt

# Rodar diretamente no terminal
Rscript -e '
  data <- read.table("combined_pvalues_filtered.txt", col.names=c("SNP", "P1", "P2"))
  data$Q <- -2*(log(data$P1) + log(data$P2))  # Estatística Q
  data$P_pleio <- pchisq(data$Q, df=2, lower.tail=FALSE)
  write.table(data, "pleiotropy_results.txt", quote=F, row.names=F)'

In [ ]:
#Opção 2: Correlação de efeitos (β)
# Extrair efeitos (OR ou β)
paste <(awk 'NR>1 {print $2, $10}' gwas_ad_results.assoc) \
      <(awk 'NR>1 {print $10}' gwas_dia_results.assoc) > combined_effects.txt

# Rodar diretamente no terminal

awk 'NF==3' combined_effects.txt > combined_effects_clean.txt

Rscript -e '
  data <- read.table("combined_effects_clean.txt", col.names=c("SNP", "OR1", "OR2"))
  cor_test <- cor.test(data$OR1, data$OR2, method="spearman")
  print(paste("Correlação de efeitos (rho):", cor_test$estimate))'

  # resultado após rodar esse script : Warning message:
# In cor.test.default(data$OR1, data$OR2, method = "spearman") :
#  Cannot compute exact p-value with ties
#[1] "Correlação de efeitos (rho): 0.00226277031660643"


In [ ]:
# Passo 4: Visualização
# Scatter plot de efeitos pleiotrópicos (R)
Rscript -e '
  # Carregar dados do arquivo corrigido
  data <- read.table("combined_effects_clean.txt", col.names=c("SNP", "OR1", "OR2"))

  # Passo 4: Visualização
  # Scatter plot de efeitos pleiotrópicos
  png("pleiotropy_scatter.png")
  plot(data$OR1, data$OR2,
       xlab="OR Alzheimer", ylab="OR Diabetes type2",
       main="Efeitos Pleiotrópicos")
  abline(h=1, v=1, lty=2, col="red")
  dev.off()
'

SyntaxError: invalid syntax (<ipython-input-4-12c14c07c875>, line 4)

In [ ]:
# Manhattan plot combinado
Rscript -e '
  # Carregar pacote qqman
  library(qqman)

  # Carregar dados GWAS
  gwas_alzheimer <- read.table("gwas_ad_results.assoc", header=TRUE)
  gwas_diabetes_type2 <- read.table("gwas_dia_results.assoc", header=TRUE)

  # Carregar SNPs pleiotrópicos
  snps_pleiotropicos <- read.table("snps_pleiotropicos.txt", header=FALSE)$V1

  # Criar PDF com dois Manhattan plots
  pdf("manhattan_plots_combined.pdf", width=12, height=10)
  par(mfrow=c(2,1))

  # Manhattan plot para Alzheimer
  manhattan(gwas_alzheimer, highlight=snps_pleiotropicos, col=c("blue4", "skyblue"),
           main="Manhattan Plot - Alzheimer")

  # Manhattan plot para Diabetes tipo 2
  manhattan(gwas_diabetes_type2, highlight=snps_pleiotropicos, col=c("red4", "indianred1"),
           main="Manhattan Plot - Type 2 diabetes")

  dev.off()

  # Informar ao usuário
  cat("Plots gerados no arquivo manhattan_plots_combined.pdf\n")
'

SyntaxError: invalid syntax (<ipython-input-5-ebe84cfb921a>, line 2)

In [ ]:
# Passo 5: Anotação funcional (opcional)
# Usar ANNOVAR ou VEP para anotar SNPs pleiotrópicos
annotate_variants.pl snps_pleiotropicos.txt -out pleio_annotated

 Dicas importantes

    Controle de FDR: Aplique correção para múltiplos testes (ex: Benjamini-Hochberg) nos resultados pleiotrópicos.

    Meta-análise: Se as doenças forem correlacionadas, use métodos como MTAG.

    Banco de dados: Consulte o *GWAS Catalog* para ver se seus SNPs já foram reportados como pleiotrópicos.


Observação:
FID  IID  PHENO
ID1  ID1  2     # Caso (doente)
ID2  ID2  1     # Controle (saudável)
ID3  ID3  -9    # Missing (opcional)

outras opçoes:

FID  IID  PHENO  IDADE  SEXO
ID1  ID1  2      45     1
ID2  ID2  1      30     2

Analisando o arquivo pleiotropy_results.txt que você obteve, posso explicar o significado das colunas:
Coluna 1: ID do SNP (por exemplo, "78196:G")
Coluna 2 (P1): P-valor da associação com Alzheimer (do arquivo gwas_ad_results.assoc)
Coluna 3 (P2): P-valor da associação com Diabetes tipo 2 (do arquivo gwas_dia_results.assoc)
Coluna 4 (Q): Estatística Q de Fisher combinada (-2*(log(P1) + log(P2)))
Coluna 5 (P_pleio): P-valor do teste de pleiotropia (significância da estatística Q)
Com esses dados, você pode criar vários tipos de gráficos informativos:

Gráfico de Manhattan para pleiotropia:



In [ ]:
# Gráfico de Manhattan para pleiotropia:
Rscript -e '
  data <- read.table("pleiotropy_results.txt", header=TRUE)
  # Extrair cromossomo e posição do ID do SNP
  data$CHR <- as.numeric(sapply(strsplit(as.character(data$SNP), ":"), function(x) x[1]))
  data$BP <- as.numeric(sapply(strsplit(as.character(data$SNP), ":"), function(x) x[2]))
  # Criar Manhattan plot
  library(qqman)
  png("pleiotropy_manhattan.png", width=1000, height=600, res=100)
  manhattan(data, chr="CHR", bp="BP", p="P_pleio",
            main="Manhattan Plot - Efeitos Pleiotrópicos",
            suggestiveline=-log10(0.05), genomewideline=-log10(0.001))
  dev.off()
'

In [ ]:
# Gráfico de Dispersão dos p-valores:
Rscript -e '
  data <- read.table("pleiotropy_results.txt", header=TRUE)
  png("pvalue_scatter.png", width=800, height=800, res=100)
  plot(-log10(data$P1), -log10(data$P2),
       xlab="-log10(P-valor Alzheimer)", ylab="-log10(P-valor Type2 Diabetes)",
       main="Significance Comparison between Phenotypes",
       pch=19, col=ifelse(data$P_pleio < 0.05, "red", "black"))
  abline(h=-log10(0.05), v=-log10(0.05), lty=2, col="blue")
  legend("topright", legend=c("Pleiotropy (p<0.05)", "Not significant"),
         col=c("red", "black"), pch=19)
  dev.off()
'

In [ ]:

# Volcano plot para pleiotropia:
Rscript -e '
  data <- read.table("pleiotropy_results.txt", header=TRUE)
  png("pleiotropy_volcano.png", width=800, height=800, res=100)
  plot(data$Q, -log10(data$P_pleio),
       xlab="Estatística Q", ylab="-log10(P-valor pleiotropia)",
       main="Volcano Plot - Pleiotropia",
       pch=19, col=ifelse(data$P_pleio < 0.05, "red", "black"))
  abline(h=-log10(0.05), lty=2, col="blue")
  legend("topright", legend=c("Pleiotropia (p<0.05)", "Não significativo"),
         col=c("red", "black"), pch=19)
  dev.off()
'

mais graficos:

In [ ]:
Rscript -e '
  data <- read.table("pleiotropy_results.txt", header=TRUE)
  png("pleiotropy_pval_histogram.png", width=800, height=600, res=100)

  # Criar histograma dos p-valores
  hist(data$P_pleio,
       breaks = 20,  # Número de barras
       xlab = "P-valor de pleiotropia",
       ylab = "Frequência",
       main = "Distribuição dos P-valores de Pleiotropia",
       col = "skyblue",
       border = "white")

  # Adicionar linha de significância
  abline(v = 0.05, col = "red", lty = 2, lwd = 2)
  legend("topright", legend = "Limite de significância (p < 0.05)",
         col = "red", lty = 2, lwd = 2)

  dev.off()
'

In [ ]:
Rscript -e '
  data <- read.table("pleiotropy_results.txt", header=TRUE)
  png("pleiotropy_pval_histogram.png", width=800, height=600, res=100)

  # Separar p-valores significativos e não significativos
  sig <- data$P_pleio < 0.05

  hist(data$P_pleio[!sig],
       breaks = 20,
       xlab = "P-valor de pleiotropia",
       ylab = "Frequência",
       main = "Distribuição dos P-valores de Pleiotropia",
       col = "gray",
       border = "white")

  hist(data$P_pleio[sig],
       breaks = 20,
       col = "red",
       border = "white",
       add = TRUE)

  abline(v = 0.05, col = "blue", lty = 2, lwd = 2)
  legend("topright",
         legend = c("P < 0.05", "P ≥ 0.05"),
         fill = c("red", "gray"))

  dev.off()
'

In [ ]:
Rscript -e '
  data <- read.table("pleiotropy_results.txt", header=TRUE)
  png("pleiotropy_qqplot.png", width=800, height=800, res=100)

  qqplot(-log10(ppoints(length(data$P_pleio))),
         -log10(sort(data$P_pleio)),
         xlab = "Valores esperados (-log10 P)",
         ylab = "Valores observados (-log10 P)",
         main = "QQ-plot dos P-valores de Pleiotropia")
  abline(0, 1, col = "red")

  dev.off()
'